In [36]:
#导入
from datasets import Dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer


In [2]:
#导入数据集
ds = Dataset.load_from_disk("./alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [12]:
#数据预处理
tokenizer = AutoTokenizer.from_pretrained("D:/Hugging Face Hub/bloom-389m-zh")

In [13]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [14]:
tokenizer_ds = ds.map(process_func,remove_columns=ds.column_names)

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

In [19]:
#创建模型
model = AutoModelForCausalLM.from_pretrained("D:/Hugging Face Hub/bloom-389m-zh")

In [ ]:
#prefix-tuning
from peft import PrefixTuningConfig,TaskType,get_peft_model
config = PrefixTuningConfig(task_type=TaskType,num_virtual_tokens=10,prefix_projection=False)
model = get_peft_model(model,config)

In [32]:
#配置训练参数
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    #gradient_accumulation_steps=2,
    logging_steps=100,
    num_train_epochs=1
)

In [33]:
#配置训练器
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenizer_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

C:\Users\wangyuncan\AppData\Local\Temp\ipykernel_24280\2553328819.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
#训练
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 166.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 5.17 GiB is allocated by PyTorch, and 84.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [35]:
#预测
from transformers import pipeline
pipe = pipeline("text-generation",model = model,tokenizer=tokenizer,device=0)

Device set to use cuda:0
